In [26]:
# namedtupleでタプルを作る機能を読み込み
#タプルとは：複数の値をひとまとめ、変更が不可な特徴がある、値の順番ある
from collections import namedtuple
import random

random.seed(7)

Item = namedtuple("item",("name","weight","price"))

num = 15
item_list = []
max_weight = 5
max_price = 50

#1から50の整数リストを作成
price_list = list(range(1,max_price+1))
#順番をランダムに並び替える
random.shuffle(price_list)


for i in range(num):
  # w に1から5のランダム値を決める
  w = random.randint(1,max_weight)

  #pop()を使用し、price_listの末尾の値を取り出す
  #pop(数字)を入れると位置を指定して値を取り出せる
  item = Item(i,w,price_list.pop())

  #appendでitem_listに追加
  item_list.append(item)

#-----------------------------ナップサックclass---------------------------------
#ナップサックを作る
#「容量が決まったカバンに、アイテムを入れていく」ための仕組みを作っている

class Knapsack:

    #（初期化）

    #__init__ はコンストラクタ:インスタンスが作られた瞬間に呼ばれる初期化メソッド
    #selfインスタンス自身,size はコンストラクタの引数
    def __init__(self, size):
      #インスタンス変数に代入self は「そのインスタンス自身」
      #インスタンス:設計図から作られた実体（オブジェクト）
        self.size = size
        self.weight = 0
        self.value = 0
        self.items = []

    #（アイテムを追加）

    def append(self, item):
        #has_room_for(item) が False なら容量オーバー
        if not self.has_room_for(item):
            raise ValueError("Over weight")

        self.items.append(item)
        self.weight += item.weight
        self.value += item.price

       #has_room_for（入るか判定）
    def has_room_for(self, item):
        return self.size >= self.weight + item.weight

    #表示
    def __str__(self):
        return "weight {} kg / value {} 0,000 Yen".format(self.weight, self.value)

#-------------------------------------------------------------------------------

#----------------------------------------貪欲法---------------------------------
#「価値 ÷ 重さ（コスパ）が良いアイテムから順に詰める」速くて実装が簡単というメリットがある。
#入るだけ入れる,入らなくなったら終了

def greedy(items,size_limit):

  #x.price / x.weight→ 価値 / 重さ（コスパ） を計算
  #reverse=True→ コスパが高い順（降順）に並べる ※Python の組み込み関数
  sorted_item_list = sorted(items,key=lambda x:x.price/x.weight,reverse=True)

  #容量 size_limit の空のナップサックを作る。
  my_knapsack = Knapsack(size_limit)

  #try/except:
  #入る → appendが成功
  #入らない → ValueError が出る → except に入り、continue でスキップ
  for v in sorted_item_list:
    try:
       my_knapsack.append(v)
    except ValueError:
        continue
  return my_knapsack

#-------------------------------------------------------------------------------

#------------------------------ブルートフォース---------------------------------
#「全部の組み合わせを試して、最も価値が高いものを選ぶ」という力技。組み合わせは 2ⁿ 通り
# メリット：必ず最適解が見つかる（100% 正しい）
# デメリット：とにかく遅い

import itertools

def brute_force(items, size_limit):
    #candidate今まで見つけた中で 最も価値が高いナップサックを保存する変数
    candidate = None

    # 全パターンを生成する「入れない(0) / 入れる(1)」の組み合わせを作る
    #repeat=len(items) なので、アイテム数が20なら 20ビットの0/1列を作る

    #itertools.product複数のリストの「直積（組み合わせ）」を作る関数。

    for pattern in itertools.product((0, 1), repeat=len(items)):

       #pattern の i 番目が 1 なら、そのアイテムを入れる 0 なら入れない
        my_box = []

        #enumerate:リストなどをループするときに、インデックス（番号）値（要素）の両方を同時に取り出す。
        for i, val in enumerate(pattern):
            if val:
                # item_list ではなく items を使う
                my_box.append(items[i])

        #重さチェック 選んだアイテムの重さ合計を計算 容量オーバーならスキップ
        w = sum(item.weight for item in my_box)
        if w > size_limit:
            continue

        #価値を計算して最適解を更新
        value = sum(item.price for item in my_box)

        #今までの最高価値より高ければ candidate を更新
        if candidate is None or value > candidate.value:
            # 新しいナップサックを作って詰め直す
            candidate = Knapsack(size_limit)

            for v in my_box:
                candidate.append(v)

    return candidate


#-------------------------------------------------------------------------------

#------------------------------DP動的計画法-------------------------------------

def dp(items, size_limit):
    # アイテム数
    n = len(items)

    # DPテーブル（価値を保存する）
    # table[i][j] = i番目までのアイテムを使って、重さj以下で得られる最大価値
    table = [[0] * (size_limit + 1) for _ in range(n + 1)]

    # 復元用フラグ（そのマスで「入れたかどうか」を記録）
    flag = [[False] * (size_limit + 1) for _ in range(n + 1)]

    # i = 1 から n まで（アイテムを1つずつ増やしていく）
    for i in range(1, n + 1):
        target = items[i - 1]   # 今見ているアイテム
        w = target.weight       # そのアイテムの重さ

        # j = 1 から size_limit まで（ナップサックの容量）
        for j in range(1, size_limit + 1):

            # yellow：そのアイテムを「入れない」場合の価値
            yellow = table[i - 1][j]
            table[i][j] = yellow  # とりあえず入れない場合で初期化

            # 重さが j を超えるなら入れられない
            if w > j:
                continue

            # pink：そのアイテムを「入れる」場合の価値
            # ＝「前の行の j-w の価値」＋「このアイテムの価値」
            pink = table[i - 1][j - w] + target.price

            # include_this：入れた場合の価値
            include_this = pink

            # yellow（入れない）と include_this（入れる）の大きい方を採用
            table[i][j] = max(yellow, include_this)

            # 入れた方が価値が大きいならフラグを立てる
            flag[i][j] = include_this > yellow

    #----------------------------------------
    # ここから「どのアイテムを入れたか」を復元する
    #----------------------------------------

    i = n
    j = size_limit
    my_knapsack = Knapsack(size_limit)

    # i を逆からたどりながら、flag を見て復元する
    while i > 0 and j > 0:
        if flag[i][j]:  # このマスで「入れた」なら
            item = items[i - 1]
            my_knapsack.append(item)  # ナップサックに追加
            j -= item.weight          # 重さ分だけ j を戻す
        i -= 1  # 次のアイテムへ（上の行へ戻る）

    return my_knapsack

#-------------------------------------------------------------------------------





print(*item_list , sep="\n")

print("----------------------------------------貪欲法---------------------------------")

knap_g = greedy(item_list,30)

print(knap_g)

print(*knap_g.items ,sep = "\n")

print("------------------------------ブルートフォース---------------------------------")

knap_bf = brute_force(item_list,30)

print(knap_bf)

print(*knap_bf.items ,sep = "\n")

print("------------------------------DP動的計画法-------------------------------------")

knap_bf = dp(item_list,30)

print(knap_bf)

print(*knap_bf.items ,sep = "\n")

print("-------------------------------------------------------------------------------")



item(name=0, weight=3, price=21)
item(name=1, weight=1, price=10)
item(name=2, weight=5, price=26)
item(name=3, weight=1, price=42)
item(name=4, weight=5, price=4)
item(name=5, weight=1, price=5)
item(name=6, weight=5, price=35)
item(name=7, weight=2, price=7)
item(name=8, weight=4, price=24)
item(name=9, weight=5, price=38)
item(name=10, weight=4, price=46)
item(name=11, weight=3, price=33)
item(name=12, weight=4, price=14)
item(name=13, weight=5, price=3)
item(name=14, weight=4, price=6)
----------------------------------------貪欲法---------------------------------
weight 29 kg / value 261 0,000 Yen
item(name=3, weight=1, price=42)
item(name=10, weight=4, price=46)
item(name=11, weight=3, price=33)
item(name=1, weight=1, price=10)
item(name=9, weight=5, price=38)
item(name=0, weight=3, price=21)
item(name=6, weight=5, price=35)
item(name=8, weight=4, price=24)
item(name=5, weight=1, price=5)
item(name=7, weight=2, price=7)
------------------------------ブルートフォース-------------------------

In [ ]:
from collections import namedtuple
import random
import itertools

random.seed(7)

Item = namedtuple("item", ("name", "weight", "price"))

num = 15
item_list = []
max_weight = 5
max_price = 50

price_list = list(range(1, max_price + 1))
random.shuffle(price_list)

for i in range(num):
    w = random.randint(1, max_weight)
    item = Item(i, w, price_list.pop())
    item_list.append(item)


class Knapsack:
    def __init__(self, size):
        self.size = size
        self.weight = 0
        self.value = 0
        self.items = []

    def append(self, item):
        if not self.has_room_for(item):
            raise ValueError("Over weight")
        self.items.append(item)
        self.weight += item.weight
        self.value += item.price

    def has_room_for(self, item):
        return self.size >= self.weight + item.weight

    def __str__(self):
        return "weight {} kg / value {} 0,000 Yen".format(self.weight, self.value)


def greedy(items, size_limit):
    sorted_item_list = sorted(items, key=lambda x: x.price / x.weight, reverse=True)
    my_knapsack = Knapsack(size_limit)
    for v in sorted_item_list:
        try:
            my_knapsack.append(v)
        except ValueError:
            continue
    return my_knapsack


def brute_force(items, size_limit):
    candidate = None
    for pattern in itertools.product((0, 1), repeat=len(items)):
        my_box = []
        for i, val in enumerate(pattern):
            if val:
                my_box.append(items[i])
        w = sum(item.weight for item in my_box)
        if w > size_limit:
            continue
        value = sum(item.price for item in my_box)
        if candidate is None or value > candidate.value:
            candidate = Knapsack(size_limit)
            for v in my_box:
                candidate.append(v)
    return candidate


def dp(items, size_limit):
    n = len(items)
    table = [[0] * (size_limit + 1) for _ in range(n + 1)]
    flag = [[False] * (size_limit + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        target = items[i - 1]
        w = target.weight
        for j in range(1, size_limit + 1):
            yellow = table[i - 1][j]
            table[i][j] = yellow
            if w > j:
                continue
            pink = table[i - 1][j - w] + target.price
            include_this = pink
            table[i][j] = max(yellow, include_this)
            flag[i][j] = include_this > yellow

    i = n
    j = size_limit
    my_knapsack = Knapsack(size_limit)

    while i > 0 and j > 0:
        if flag[i][j]:
            item = items[i - 1]
            my_knapsack.append(item)
            j -= item.weight
        i -= 1

    return my_knapsack


print(*item_list, sep="\n")

print("----------------------------------------貪欲法---------------------------------")
knap_g = greedy(item_list, 30)
print(knap_g)
print(*knap_g.items, sep="\n")

print("------------------------------ブルートフォース---------------------------------")
knap_bf = brute_force(item_list, 30)
print(knap_bf)
print(*knap_bf.items, sep="\n")

print("------------------------------DP動的計画法-------------------------------------")
knap_dp = dp(item_list, 30)
print(knap_dp)
print(*knap_dp.items, sep="\n")

print("-------------------------------------------------------------------------------")
